In [27]:
from power_grid_model import PowerGridModel, CalculationMethod
from power_grid_model.utils import json_deserialize_from_file
import numpy as np


In [31]:
def calculate_all_source_sk(start_sk_log, end_sk_log, method: CalculationMethod = CalculationMethod.newton_raphson):
    n_steps = end_sk_log - start_sk_log
    source_p = np.zeros(n_steps, dtype=np.float64)
    source_q = np.zeros(n_steps, dtype=np.float64)
    node_voltage_pu = np.zeros(n_steps, dtype=np.float64)
    line_p = np.zeros(n_steps, dtype=np.float64)
    line_q = np.zeros(n_steps, dtype=np.float64)

    input_data = json_deserialize_from_file("grid.json")

    for i, sk_log in enumerate(range(start_sk_log, end_sk_log)):
        sk = float(10 ** sk_log)
        input_data["source"]["sk"] = sk
        model = PowerGridModel(input_data)
        result = model.calculate_power_flow()
        source_p[i] = result["source"]["p"][0]
        source_q[i] = result["source"]["q"][0]
        node_voltage_pu[i] = result["node"]["u_pu"][0]
        line_p[i] = result["line"]["p_from"][0]
        line_q[i] = result["line"]["q_from"][0]
    
    return {
        "source_p": source_p, 
        "source_q": source_q, 
        "node_voltage_pu": node_voltage_pu, 
        "line_p": line_p, 
        "line_q": line_q,
        "sk": np.array([float(10 ** sk_log) for sk_log in range(start_sk_log, end_sk_log)])
    }


In [34]:
pgm_calculation = calculate_all_source_sk(8, 40, CalculationMethod.newton_raphson)